<a href="https://colab.research.google.com/github/jserrataylor/mlflow/blob/main/semaforo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EL SIGUIENTE SCRIPT DESARROLLA UN MODELO PARA PREDECIR EL PATRÓN DE TRAFICO EN UN SEMAFORO POR HORA.

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Cargar y procesar los datos
data = pd.read_csv("traffic_data.csv")
data["timestamp"] = pd.to_datetime(data["timestamp"])
data = data.set_index("timestamp")

# Preprocesamiento de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(data_scaled) * 0.8)
train, test = data_scaled[0:train_size, :], data_scaled[train_size:len(data_scaled), :]

# Función para crear conjuntos de datos de series temporales
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        dataX.append(dataset[i:(i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Crear y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# Realizar predicciones
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Transformar las predicciones de vuelta a la escala original
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# Calcular el error cuadrático medio (MSE)
trainScore = mean_squared_error(trainY[0], trainPredict[:, 0])
testScore = mean_squared_error(testY[0], testPredict[:, 0])

print(f"Train Score: {trainScore:.2f} MSE")
print(f"Test Score: {testScore:.2f} MSE")

# Graficar las predicciones
plt.plot(data.index[:len(trainPredict)], trainPredict, label="Predicciones de entrenamiento")
plt.plot(data.index[len(trainPredict):len(trainPredict) + len(testPredict)], testPredict, label="Predicciones de prueba")
plt.plot(data.index, scaler.inverse_transform(data_scaled), label="Datos reales")
plt.legend()
plt.show()

def predict_traffic_at_hour(hour):
    if not 0 <= hour <= 23:
        raise ValueError("La hora debe estar entre 0 y 23")
    
    hour_index = data.index[data.index.hour == hour][-1]
    hour_index_position = data.index.get_loc(hour_index)

    last_values = data_scaled[hour_index_position - look_back + 1:hour_index_position + 1]
    last_values = np.reshape(last_values, (1, 1, look_back))
    prediction = model.predict(last_values)
    prediction = scaler.inverse_transform(prediction)
    return prediction[0, 0]

In [32]:
# Ejemplo de uso del modelo para predecir el tráfico y ajustar el tiempo de espera del semáforo
input_hour = int(input("Ingrese la hora (0-23) para la que desea predecir el tráfico: "))
predicted_traffic = predict_traffic_at_hour(input_hour)
print(f"El tráfico predicho para la hora {input_hour} es: {predicted_traffic:.0f} vehículos")

Ingrese la hora (0-23) para la que desea predecir el tráfico: 16
1/1 [==============================] - 0s 17ms/step
El tráfico predicho para la hora 16 es: 53 vehículos


## PREDICCIÓN DE TRAFICO POR HORA EN UN SEMAFORO QUE INCLUYE LAS HORA Y LOS MINUTOS, PERO NECESITA DATOS DE ENTRENAMIENTO QUE INCLUYAN MINUTOS.

In [ ]:

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Cargar y procesar los datos
data = pd.read_csv("traffic_data.csv")
data["timestamp"] = pd.to_datetime(data["timestamp"])
data = data.set_index("timestamp")

# Preprocesamiento de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(data_scaled) * 0.8)
train, test = data_scaled[0:train_size, :], data_scaled[train_size:len(data_scaled), :]

# Función para crear conjuntos de datos de series temporales
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        dataX.append(dataset[i:(i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Crear y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# Realizar predicciones
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Transformar las predicciones de vuelta a la escala original
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# Calcular el error cuadrático medio (MSE)
trainScore = mean_squared_error(trainY[0], trainPredict[:, 0])
testScore = mean_squared_error(testY[0], testPredict[:, 0])

print(f"Train Score: {trainScore:.2f} MSE")
print(f"Test Score: {testScore:.2f} MSE")

# Graficar las predicciones
plt.plot(data.index[:len(trainPredict)], trainPredict, label="Predicciones de entrenamiento")
plt.plot(data.index[len(trainPredict):len(trainPredict) + len(testPredict)], testPredict, label="Predicciones de prueba")
plt.plot(data.index, scaler.inverse_transform(data_scaled), label="Datos reales")
plt.legend()
plt.show()

def predict_traffic_at_time(hour, minute):
    if not 0 <= hour <= 23:
        raise ValueError("La hora debe estar entre 0 y 23")
    if not 0 <= minute <= 59:
        raise ValueError("Los minutos deben estar entre 0 y 59")
    
    minute_index = data.index[(data.index.hour == hour) & (data.index.minute == minute)][-1]
    minute_index_position = data.index.get_loc(minute_index)

    last_values = data_scaled[minute_index_position - look_back + 1:minute_index_position + 1]
    last_values = np.reshape(last_values, (1, 1, look_back))
    prediction = model.predict(last_values)
    prediction = scaler.inverse_transform(prediction)
    return prediction[0, 0]

# Ejemplo de uso del modelo para predecir el tráfico y ajustar el tiempo de espera del semáforo
input_time = input("Ingrese la hora y los minutos en formato hh:mm para predecir el tráfico: ")
input_hour, input_minute = map(int, input_time.split(':'))

predicted_traffic = predict_traffic_at_time(input_hour, input_minute)
print(f"El tráfico predicho para las {input_hour}:{input_minute:02d} es: {predicted_traffic:.0f} vehículos")

# Aquí puedes ajustar el tiempo de espera del semáforo según la predicción del modelo
